首先运行下基本的命令，检查下环境是否搭建成功

In [1]:
import tensorflow as tf

hello = tf.constant('Hello Tensorflow!')
with tf.Session() as sess:
    print(sess.run(hello))

b'Hello Tensorflow!'


In [7]:
# Tensorflow搭建成功，再试一下通过VGG19预测一张图片
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from imagenet_utils import preprocess_input, decode_predictions


model = VGG19(weights='imagenet')

img_path = 'data/train/cat/cat.0.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print('Predicted:', decode_predictions(preds))



ImportError: Could not import PIL.Image. The use of `array_to_img` requires PIL.